In [ ]:
# !pip install deepdish
!pip install tf_slim

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('./drive/My Drive/project/code')
sys.path
from function_2 import *  # for input size 512
from function_1 import *  # for input size 128
import numpy as np
import matplotlib.pyplot as plt
# import deepdish as dd
import tensorflow as tf
import tf_slim as slim
import os
tf.compat.v1.disable_eager_execution()
# tf.config.set_soft_device_placement(True)
# tf.config.experimental.list_physical_devices('GPU')

In [ ]:
# This parameter setting is referred from https://github.com/robertocalandra/the-feeling-of-success/tree/master/manu_sawyer/src/tensorflow_model_is_gripping
parm = gel_fulldata_v5()

In [ ]:
init_path='./drive/My Drive/project/vgg_16.ckpt'
os.putenv('CUDA_VISIBLE_DEVICES', ','.join(map(str, [0,1])))
mkdir(parm.resdir)
mkdir(parm.dsdir)
mkdir(parm.train_dir)
use_reg = True
restore = False
update_top_only=False
tf.compat.v1.reset_default_graph()
config = tf.compat.v1.ConfigProto(allow_soft_placement = True)
tf.Graph().as_default()
# with tf.Graph().as_default(), tf.device('/cpu:0'), tf.compat.v1.Session(config = config) as sess:
with tf.Graph().as_default(), tf.device('/device:GPU:0'), tf.compat.v1.Session(config = config) as sess:

    inputs = read_data(parm,sess)
    global_step = tf.compat.v1.get_variable('global_step', [], initializer = 
                              tf.constant_initializer(0), trainable = False)

    lr = parm.base_lr * parm.lr_gamma**(global_step // parm.step_size)

    if parm.opt_method == 'adam':
      opt = tf.compat.v1.train.AdamOptimizer(lr)
    elif parm.opt_method == 'momentum':
      opt = tf.compat.v1.train.MomentumOptimizer(lr, 0.9)

    with tf.device('/device:GPU:0'):
      label = inputs['is_gripping']

      logits = make_model(inputs, parm, update_top_only = update_top_only, train = True, reuse = False)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits = logits, labels = label)
      loss = tf.reduce_mean(loss)
      if use_reg:
        reg_losses = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES)
        print ('Number of regularization losses:', len(reg_losses))
        loss = loss + tf.add_n(reg_losses)
      eq = tf.equal(tf.argmax(logits, 1), label)
      acc = tf.reduce_mean(tf.cast(eq, tf.float32))

    train_op = opt.minimize(loss, global_step = global_step)

    bn_ups = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
    print ('Batch norm updates:', len(bn_ups))
    train_op = tf.group(train_op, *bn_ups)

    sess.run(tf.compat.v1.global_variables_initializer())
    var_list = slim.get_variables_to_restore()
    re=var_list

    if restore:
      exclude = ['Adam', 'beta1_power', 'beta2_power', 'Momentum', 'global_step' , 'fc6_']
    else:
      exclude = ['Adam', 'beta1_power', 'beta2_power', 'Momentum', 'global_step', 'logits', 'fc8', 'fc6_', 'fc7_', 'conv6']

    var_list = [x for x in var_list if \
                not any(name in x.name for name in exclude)]

    if restore:
      tf.compat.v1.train.Saver(var_list).restore(sess, pj('./drive/My Drive/project/result/gel-fulldata-v5/training', 'net.tf-set123'))
      # tf.compat.v1.train.Saver(var_list).restore(sess, tf.train.latest_checkpoint(parm.train_dir))
    else:
      for base in ['im', 'depth', 'gel']:
          print ('Restoring:', base)
          mapping = {}
          for v in var_list:
            start = '%s_vgg16/' % base
            if v.name.startswith(start):
              vgg_name = v.name.replace(start, 'vgg_16/')
              vgg_name = vgg_name[:-2]
              print (vgg_name, '->', v.name)
              mapping[vgg_name] = v
          if len(mapping):
            tf.compat.v1.train.Saver(mapping).restore(sess, init_path)

    tf.compat.v1.train.start_queue_runners(sess = sess)
    for i in range(parm.train_iters):
      step = int(sess.run(global_step))
      # print(step)
      # if (step == 10 or step % 100 == 0) or step == parm.train_iters - 1:
      if step % 1000 == 0 or step == parm.train_iters - 1:
        check_path = os.path.join(mkdir(parm.train_dir), 'net.tf')
        print ('Saving:', check_path)
        vs = slim.get_model_variables()
        tf.compat.v1.train.Saver(vs).save(sess, check_path, global_step = global_step)
      if step > parm.train_iters:
        break

      _, lr_val, loss_val, acc_val = sess.run([train_op, lr, loss, acc])

      if step % 10 == 0:
        print ('Iteration %d,' % step, 'lr = ', lr_val, \
              'loss:', moving_avg('loss', loss_val), 'acc:', moving_avg('acc', acc_val))
        sys.stdout.flush()

Tf files: ['./drive/My Drive/project/result/train128_set2.tf', './drive/My Drive/project/result/train128_set3.tf', './drive/My Drive/project/result/train128_set4.tf']
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Prefer Dataset.range instead.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions fo